In [1]:
import cfbd
import json
import matplotlib.pyplot as plt
from datetime import datetime
import numpy as np
import json
import os 

configuration = cfbd.Configuration()
configuration.api_key['Authorization'] = 'j5Z7/ASYjq4l6PMJFJpY7WxwO5BtbY/U2TFBOVdziGKRvjI8WajGUnVRiUBGrIX8'
configuration.api_key_prefix['Authorization'] = 'Bearer'

games_api_instance = cfbd.GamesApi(cfbd.ApiClient(configuration))
metrics_api_instance = cfbd.MetricsApi(cfbd.ApiClient(configuration))
plays_api_instance = cfbd.PlaysApi(cfbd.ApiClient(configuration))

In [2]:
def clean_teamname(team):
    # Special cases
    if team == 'olemiss':
        return 'ole miss'
    if team == 'sanjosestate':
        return 'san josé state'
    if team == 'usf':
        return 'south florida'
    
    # Ending token (georgia tech)     
    if team[-5:] == 'state':
        team = team[:-5] + ' state'
    if team[-4:] == 'tech':
        team = team[:-4] + ' tech'
    if team[-2:] == 'am':
        team = team[:-2] + ' a&m'
    if team[-4:] == 'dame':
        team = team[:-4] + ' dame'
    if team[-7:] == 'college':
        team = team[:-7] + ' college'
    
    # Beginning token (north dakota)     
    if team[:5] == 'north':
        team = 'north ' + team[5:]
    if team[:5] == 'south':
        team = 'south ' + team[5:]
    if team[:7] == 'central':
        team = 'central ' + team[7:]
    if team[:4] == 'west':
        team = 'west ' + team[4:]
    return team

In [30]:
# Find top 100 files by line length
# From filename, get team 1 and team 2
# From random comment, get a timestamp --> week number

name_of_dir = 'threads'
  
# Storing list of all files 
# in the given directory in list_of_files 
list_of_files = filter( lambda x: os.path.isfile 
                       (os.path.join(name_of_dir, x)), 
                        os.listdir(name_of_dir) ) 
  
# Sort list of file names by size  
list_of_files = sorted( list_of_files, 
                        key =  lambda x: os.stat 
                       (os.path.join(name_of_dir, x)).st_size)

list_of_files.reverse()
list_of_files = list_of_files[99:102]
# print(len(list_of_files))
print(list_of_files)

team1 = None
team2 = None
matchups = []

for filename in list_of_files:
    teams = filename.split("_")
    team1 = clean_teamname(teams[0])
    team2 = clean_teamname(teams[1][:-4])
    matchups.append((team1, team2, filename[:-4]))
# print(matchups)

['alabama_olemiss.csv', 'georgia_kentucky.csv', 'missouri_lsu.csv']


In [22]:
YEAR = 2023

# TODO: timezones
for team1, team2, filename in matchups:
    
    games = None
    season_type = 'regular'
    # Special case: oregon, washington played twice     
    if team1=='oregon' and team2=='washington':
        # PAC12 Championship game
        games = games_api_instance.get_games(year=YEAR, home=team2, away=team1, week=14, season_type=season_type)
    elif team1=='washington' and team2=='oregon':
        # Regular season game at washington
        games = games_api_instance.get_games(year=YEAR, home=team1, away=team2, week=7, season_type=season_type)
    else:
        games = games_api_instance.get_games(year=YEAR, home=team1, away=team2)
        if len(games) == 0:
            games = games_api_instance.get_games(year=YEAR, home=team2, away=team1)
        if len(games) == 0:
            season_type = 'postseason'
            games = games_api_instance.get_games(year=YEAR, home=team1, away=team2, season_type=season_type)
        if len(games) == 0:
            season_type='postseason'
            games = games_api_instance.get_games(year=YEAR, home=team2, away=team1, season_type=season_type)
    if len(games)==0:
        print('no games found')
        print(team1, team2)

    game = games[0]
    output = []
    print(f'id: {game._id} home: {game._home_team} away: {game._away_team}')
    wp = metrics_api_instance.get_win_probability_data(game_id=game._id)
    if not wp:
        print('no wp')
        print(team1, team2)
        continue
    print(f'initial win prob: {wp[0]._home_win_prob:.3f}, final: {wp[-1]._home_win_prob:.3f}')
    plays = plays_api_instance.get_plays(year=YEAR, week=game._week, team=game._home_team, season_type=season_type)
    for i in range(len(wp)):
        j = i
        while (j < len(plays) and wp[i]._play_id != plays[j]._id):
            j += 1

        dt_utc = None
        if j == len(plays) or plays[j]._wallclock is not None:
            if j == len(plays) or ():
                dt_utc = output[-1]['utc'] + 1
            else:
                dt_utc = datetime.fromisoformat(plays[j]._wallclock).astimezone(tz=None).timestamp()
            # Don't allow timestamp for a given play to be before a previous play!
            if i > 1 and dt_utc < output[-1]['utc']:
                dt_utc = output[-1]['utc'] + 1
            if (j == len(plays) or wp[i]._play_id == plays[j]._id):
                agg_play = {
                    'play_number' : i,
                    'play_id' : wp[i]._play_id,
                    'home_win_prob' : wp[i]._home_win_prob,
                    # 'wallclock' : plays[j]._wallclock,
                    'utc': int(dt_utc),
                }
                output.append(agg_play)
    with open(f'win_probs/{filename}.json', 'w') as f:
        json.dump(output, f)
#         # Plotting win percentage
#         time = [play['utc'] for play in output]
#         wp_list = [play['home_win_prob'] for play in output]
#         plot = plt.plot(time, wp_list)
#         plt.xlabel('UTC time')
#         plt.ylabel('Home Win Prob')
#         plt.title(f'(H) {game._home_team} vs (A) {game._away_team}')
#         plt.show()

#         plot = plt.plot(range(len(output)), wp_list)
#         plt.xlabel('Play Number')
#         plt.ylabel('Home Win Prob')
#         plt.title(f'(H) {game._home_team} vs (A) {game._away_team}')
#         plt.show()
#         print(game._week, game._season_type)
#         break

We have the top 102 games with the exception of two for which we were unable to retrieve the win probability. 
These two excluded games are 
1. Louisville vs Georgia Tech
2. Penn State vs West Virginia